In [1]:
import sys
import time
import random
import csv
import numpy as np

import scipy.io
from scipy.io import loadmat

import torch
import pickle

from tqdm import tqdm
from tqdm import notebook

import matplotlib
import matplotlib.pyplot as plt

import importlib
import utils
importlib.reload(utils)
TORCH_DTYPE = torch.float64 #NB: Basically all of the matrices in Spatial_GP have 1.e-7 added to the diagonal, to be changed if we want to use float64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
torch.set_default_dtype(TORCH_DTYPE)
torch.set_default_device(device)
print(f'Device is: {device}')


Using device: cuda:0 (from utils.py)
Using device: cuda:0 (from utils.py)
Device is: cuda:0


### How this works:

We choose initial training points to be considered a random set of images to present to the retina. On this set, the complete GP will be run , to find STA paramaters eps_0: center, beta: width , rho: smoothness

We run the algorithm saving the seed used for picking the training images, so that this set can be changed and different runs with different initial conditions can be averaged.

1. Import the dataset and create a total training set X,R
2. Pick the cell and the initial training points, extracted randomly. These correspond also to the number of inducing points
3. Save the seed so you can keep the initial training set, and the fitted model
3. 


### Parameters of the training

In [2]:
rand_xtilde = True # If True, xtilde (inducing points) are chosen randomly, if False, xtilde is chosen from the first ntilde images

cellid      = 8   # Choose cell
ntrain_start = 50 # Number of first training data points

kernfun     = utils.acosker # Choose kernel function

Nmstep  = 6   # Total number of M-steps iterations. 
Nestep  = 8   # Total number of E-steps iterations.
Maxiter = 10  # Iterations of the optimization algorithm comprising M and E steps

ntilde       = ntrain_start

### Import dataset and generate starting dataset

Create starting dataset on which to train with m step with ntilde = ntrain_start

In [3]:
# Open the .pkl dataset file for reading in binary mode (rb)
with open('/home/idv-eqs8-pza/IDV_code/Variational_GP/spatial_GP/Data/data2_41mixed_tr28.pkl', 'rb') as file:
    # Load the data from the file
    loaded_data = pickle.load(file)
    # loaded_data is a Dataset object from module Data with attributes "images_train, _val, _test" as well as responses

X_train = torch.tensor(loaded_data.images_train).to(device, dtype=TORCH_DTYPE) #shape (2910,108,108,1) where 108 is the number of pixels. 2910 is the amount of training points
X_val   = torch.tensor(loaded_data.images_val).to(device, dtype=TORCH_DTYPE)
X_test  = torch.tensor(loaded_data.images_test).to(device, dtype=TORCH_DTYPE) # shape (30,108,108,1) # nimages, npx, npx

R_train = torch.tensor(loaded_data.responses_train).to(device, dtype=TORCH_DTYPE) #shape (2910,41) 2910 is the amount of training data, 41 is the number of cells
R_val   = torch.tensor(loaded_data.responses_val).to(device, dtype=TORCH_DTYPE)
R_test  = torch.tensor(loaded_data.responses_test).to(device, dtype=TORCH_DTYPE) # shape (30,30,42) 30 repetitions, 30 images, 42 cells

# Create the complete dataset
X = torch.cat( (X_train, X_val), axis=0,) #shape (3160,108,108,1)
R = torch.cat( (R_train, R_val), axis=0,)
n_px_side = X.shape[1]  

# Reshape images to 1D vector  
X = torch.reshape(X, ( X.shape[0], X.shape[1]*X.shape[2])) 

# Choose a random subset of the data and save the idx, get also
torch.manual_seed(0)
arange = torch.arange(0, X.shape[0])
rndm   = torch.randint(0, X.shape[0], (ntrain_start,))
combined = torch.cat( (arange, rndm) )
unique, counts = combined.unique(return_counts=True)
remaining_idx = unique[counts==1]
start_idx = unique[counts>1]

xtilde = X[start_idx,:]
xtrain = xtilde
Xremaining = X[remaining_idx,:]

# Choose a cell
r = R[:,cellid] # shape (nt,) where nt is the number of trials

### Select cell, starting hyperparameters and firing rate parameters

In [4]:
# For details on the hyperparameters choice see one_cell_fit.ipynb
logbetaexpr = utils.fromlogbetasam_to_logbetaexpr( logbetasam=torch.tensor(5.5) )# Logbetaexpr in this code is equal to logbeta in Samuele's code. Samuele's code set logbeta to 5.5
logrhoexpr  = utils.fromlogrhosam_to_logrhoexpr( logrhosam=torch.tensor(5)) 
# logbetaexpr = torch.tensor(4.65)
# logrhoexpr = torch.tensor(4.3)
logsigma_0 = torch.tensor(0) 
sigma_0    = torch.exp(logsigma_0)
Amp        = torch.tensor(1.0) 

# Hypermarameters, if needed to be set manually
theta = {'sigma_0': sigma_0, 'eps_0x':torch.tensor(0.), 'eps_0y':torch.tensor(0.), '-2log2beta': logbetaexpr, '-log2rho2': logrhoexpr, 'Amp': Amp }
# Set the gradient of the hyperparemters to be updatable 
for key, value in theta.items():
    # to exclude a single hyperparemeters from the optimization ( to exclude them all just set nmstep=0 and dont do the M-step)
    # if key == 'Amp':
        # continue
    theta[key] = value.requires_grad_()

# If hyperparameters are set manually:
hyperparams_tuple = utils.generate_theta( x=X, r=r, n_px_side=n_px_side, display=True, **theta)
# If hyperparameters are set based on the STAs:
# hyperparams_tuple = utils.generate_theta( x=X, r=r, n_px_side=n_px_side, display=True)

# Set parameters of the firing rate function
A        = torch.tensor(0.007)
logA     = torch.log(A)
f_params = {'logA': logA, 'lambda0':torch.tensor(0.31)}

for key, value in f_params.items():
    f_params[key] = value.requires_grad_()

args = {
        'ntilde':  ntrain_start,
        'Maxiter': Maxiter,
        'Nmstep':  Nmstep,
        'Nestep':  Nestep,
        'kernfun': kernfun,
        'n_px_side': n_px_side,
        'display_prog':  False,
        'hyperparams_tuple': hyperparams_tuple,
        'f_params': f_params,
        'xtilde': xtilde,
        'm': torch.zeros( (ntilde) )
    }



updated sigma_0 to 1.0
updated eps_0x to 0.0
updated eps_0y to 0.0
updated -2log2beta to 4.806852819440055
updated -log2rho2 to 4.306852819440055
updated Amp to 1.0
 Before overloading
 Hyperparameters have been SET as  : beta = 0.05856070, rho = 0.02928035
 Samuele hyperparameters           : logbetasam = 4.9822, logrhosam = 7.0617

 After overloading
 Dict of learnable hyperparameters : sigma_0 = 1.00000000, eps_0x = 0.00000000, eps_0y = 0.00000000, -2log2beta = 4.80685282, -log2rho2 = 4.30685282, Amp = 1.00000000
 Hyperparameters from the logexpr  : beta = 0.04520382, rho = 0.08208500
 Samuele hyperparameters           : logbetasam = 5.5000, logrhosam = 5.0000


### Fit the model
And save it needed to start a new active fit

In [5]:
theta_fit, f_params_fit, m_b_fit, V_b_fit, C_fit, mask_fit, K_tilde_b_fit, K_tilde_inv_b_fit, B, values_track_fit = utils.varGP(X, r, **args)

# Save the model. All of the matrices are projected in the eigenspace of big eigenvalues of K_tilde. Indicated by _b
model_b_starting = {
    'theta': theta_fit,
    'f_params': f_params_fit,
    'm_b': m_b_fit,
    'V_b': V_b_fit,
    'C'  : C_fit,
    'mask': mask_fit,
    'K_tilde_inv_b': K_tilde_inv_b_fit,
    'K_tilde_b': K_tilde_b_fit,
    'B': B,
    'cellid' : cellid,
    'ntilde' : ntilde,
    'Maxiter': Maxiter,
    'Nmstep' : Nmstep,
    'Nestep' : Nestep,
    'kernfun': kernfun,
    'values_track': values_track_fit,
}

# Save the model
# save_pickle('pietro_model', **model)

*Iteration*: 0
 2940.8513 = logmarginal in m-step
 2924.4308 = logmarginal in m-step
 2924.2237 = logmarginal in m-step
 2924.2237 = logmarginal in m-step
 2915.6985 = logmarginal in m-step
 2915.6705 = logmarginal in m-step
 2915.6705 = logmarginal in m-step
 2914.7421 = logmarginal in m-step
 2914.7321 = logmarginal in m-step
*Iteration*: 1
 2852.9357 = logmarginal in m-step
 2831.7983 = logmarginal in m-step
 2831.5464 = logmarginal in m-step
 2831.5464 = logmarginal in m-step
 2820.8374 = logmarginal in m-step
 2820.7878 = logmarginal in m-step
 2820.7878 = logmarginal in m-step
 2818.3318 = logmarginal in m-step
 2818.3295 = logmarginal in m-step
*Iteration*: 2
 2771.0141 = logmarginal in m-step
 2746.2591 = logmarginal in m-step
 2745.8120 = logmarginal in m-step
 2745.8120 = logmarginal in m-step
 2729.6119 = logmarginal in m-step
 2729.3478 = logmarginal in m-step
 2729.3478 = logmarginal in m-step
 2718.9554 = logmarginal in m-step
 2718.7884 = logmarginal in m-step
*Iteration

### Infer mean and variance of the remaining datapoints, estimate utility of each.

In [18]:
importlib.reload(utils)

with torch.no_grad():
# reshape the remaining dataset to
# Xremaining = torch.reshape(Xremaining, (Xremaining.shape[0], Xremaining.shape[0]*Xremaining.shape[1]))

    for i in range(Xremaining.shape[0]):
        xstar = Xremaining[i,:][None,:]

        mask  = model_b_starting['mask']
        C     = model_b_starting['C']
        K_tilde      = model_b_starting['K_tilde_b']
        K_tilde_inv  = model_b_starting['K_tilde_inv_b']
        m            = model_b_starting['m_b']
        V            = model_b_starting['V_b']
        f_params_fit = model_b_starting['f_params']

        mu_star, sigma2_star = utils.lambda_moments_star(xstar[:,mask], xtilde[:,mask], C, theta, K_tilde, K_tilde_inv, m, V, B, kernfun)

        u = utils.utility(sigma2_star, mu_star)

        f = utils.mean_f_given_lambda_moments( f_params_fit, mu_star, sigma2_star )

        print(f'Utility: {u.item():.4f} mu_star: {mu_star.item():.4f} sigma2_star: {sigma2_star.item():.4f} f: {f.item():.4f}')    



Using device: cuda:0 (from utils.py)
Utility: -1408.7728 mu_star: 2.3513 sigma2_star: 5.8137 f: 0.7882
Utility: 0.1943 mu_star: -4.7287 sigma2_star: 1.9434 f: 0.6056
Utility: -7015.1259 mu_star: -9.5870 sigma2_star: 30.8665 f: 0.5163
Utility: -170.5474 mu_star: 2.9671 sigma2_star: 1.2161 f: 0.8038
Utility: -348842215.9129 mu_star: 16.2631 sigma2_star: 1.2104 f: 1.3120
Utility: -21670.9787 mu_star: 7.2853 sigma2_star: 1.3350 f: 0.9425
Utility: -54.8953 mu_star: -11.1127 sigma2_star: 25.1079 f: 0.4862
Utility: -33.3481 mu_star: -2.6128 sigma2_star: 8.8169 f: 0.6577
Utility: -3384813752.2557 mu_star: 18.0509 sigma2_star: 1.8997 f: 1.4020
Utility: -18444319589.6655 mu_star: -16.5552 sigma2_star: 72.3782 f: 0.4108
Utility: 0.0003 mu_star: -13.6413 sigma2_star: 4.9652 f: 0.4369
Utility: 0.1548 mu_star: -5.0408 sigma2_star: 1.9825 f: 0.5987
Utility: -0.0788 mu_star: -13.8682 sigma2_star: 19.7359 f: 0.4377
Utility: -10.4331 mu_star: -6.8640 sigma2_star: 14.5910 f: 0.5646
Utility: -737.7765 mu_